In [2]:
!pip install tensorflow-gpu==1.8.0 Keras==2.1.6

    100% |████████████████████████████████| 216.3MB 3.0kB/s eta 0:00:01   33% |██████████▉                     | 73.3MB 61.2MB/s eta 0:00:03    81% |██████████████████████████▏     | 177.2MB 33.9MB/s eta 0:00:02    90% |█████████████████████████████   | 196.0MB 47.6MB/s eta 0:00:01
    100% |████████████████████████████████| 348kB 1.8MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 6.4MB/s eta 0:00:01
    100% |████████████████████████████████| 3.1MB 185kB/s eta 0:00:01
    100% |████████████████████████████████| 7.1MB 100kB/s eta 0:00:01
    100% |████████████████████████████████| 9.1MB 78kB/s  eta 0:00:01
    100% |████████████████████████████████| 2.8MB 253kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 6.6MB/s eta 0:00:01
    100% |████████████████████████████████| 890kB 818kB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 2.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/ubuntu/.

In [3]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
!ls data

downloaded	processed		   test.zip  train	valid
labels.csv.zip	sample_submission.csv.zip  tmp	     train.zip


In [12]:
train_folder = "data/train"
valid_folder = "data/valid"

In [13]:

IMAGE_SIZE=224
BATCH_SIZE=16
NUM_CLASSES=120

In [16]:
!ls data/train/

affenpinscher			italian_greyhound
afghan_hound			japanese_spaniel
african_hunting_dog		keeshond
airedale			kelpie
american_staffordshire_terrier	kerry_blue_terrier
appenzeller			komondor
australian_terrier		kuvasz
basenji				labrador_retriever
basset				lakeland_terrier
beagle				leonberg
bedlington_terrier		lhasa
bernese_mountain_dog		malamute
black-and-tan_coonhound		malinois
blenheim_spaniel		maltese_dog
bloodhound			mexican_hairless
bluetick			miniature_pinscher
border_collie			miniature_poodle
border_terrier			miniature_schnauzer
borzoi				newfoundland
boston_bull			norfolk_terrier
bouvier_des_flandres		norwegian_elkhound
boxer				norwich_terrier
brabancon_griffon		old_english_sheepdog
briard				otterhound
brittany_spaniel		papillon
bull_mastiff			pekinese
cairn				pembroke
cardigan			pomeranian
chesapeake_bay_retriever	pug
chihuahua			redbone
chow				rhodesian_ridgeback
clumber				rottweiler
cocker_spaniel			saint_bernard
collie				saluki
curly-coated_retriever		samoyed
dandie_di

In [23]:

train_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                                   rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                  
                                  
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 2044 images belonging to 120 classes.
Found 2044 images belonging to 120 classes.


Found 2044 images belonging to 120 classes.


In [25]:
NUM_CLASSES = train_batches.num_classes




In [ ]:
mobile = keras.applications.mobilenet.MobileNet()



17227776/17225924 [==============================] - 1s 0us/step


In [ ]:
mobile.summary()



In [ ]:
x = mobile.layers[-6].output
predictions = Dense(NUM_CLASSES, activation='softmax')(x)


In [ ]:
model = Model(inputs=mobile.input, outputs=predictions)


In [ ]:
model.summary()


In [ ]:
model.layers[:-1]

In [ ]:
for layer in model.layers[:-2]:
    layer.trainable = False
    

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
model.fit_generator(train_generator, steps_per_epoch=train_generator.n//BATCH_SIZE, 
                    validation_data=validation_generator, validation_steps=3, epochs=15, verbose=2,workers=4, 
                   use_multiprocessing=True
                   
                   )


In [ ]:
?model.fit_generator

In [ ]:
train_generator.class_indices.keys()

In [ ]:
model.save('dog_breed.h5')

In [ ]:
with open('labels.txt', 'w') as file_handler:
    for item in train_generator.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [ ]:
tra

In [36]:
!gsutil cp /home/ubuntu/custom/deeplizrd/dog_breed.h5 gs://np-training-public/

Copying file:///home/ubuntu/custom/deeplizrd/dog_breed.h5 [Content-Type=application/octet-stream]...
/ [1 files][ 22.0 MiB/ 22.0 MiB]                                                
Operation completed over 1 objects/22.0 MiB.                                     


In [37]:
!pwd

/home/ubuntu/custom/deeplizrd


In [ ]:

!ls /home/ubuntu/.kaggle/competitions/dog-breed-identification/train_folder/

